In [1]:
import nltk 
import pandas as pd 
import numpy as np
import tensorflow as tf

2026-01-20 11:10:31.155056: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-20 11:10:31.192300: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-20 11:10:32.093002: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from tensorflow.keras import mixed_precision

# This creates the policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print("✅ Mixed Precision enabled! Memory usage will drop significantly.")

✅ Mixed Precision enabled! Memory usage will drop significantly.


In [3]:
# Load the dataset 
data = pd.read_csv('english_news_dataset.csv')

In [4]:
data.head()

,Headline,Content,News Categories,Date
0,Congress leader Baljinder Singh shot dead at h...,Congress leader Baljinder Singh was shot dead ...,['national'],19-09-2023
1,17-year-old girl preparing for NEET dies by su...,Another NEET aspirant died by suicide in Rajas...,['national'],19-09-2023
2,Hampers to welcome MPs in new Parliament tomor...,In order to mark the first-ever working day of...,['national'],19-09-2023
3,"Only 10% women lawmakers in RS, while only 14%...","Congress President Mallikarjun Kharge, while s...",['national'],19-09-2023
4,"Ganesh temple decorated with notes, coins wort...",The Sri Sathya Ganapathi Temple in Bengaluru a...,['national'],19-09-2023


In [5]:
data = data.drop(columns=['News Categories','Date'],axis=1)
data.head()

,Headline,Content
0,Congress leader Baljinder Singh shot dead at h...,Congress leader Baljinder Singh was shot dead ...
1,17-year-old girl preparing for NEET dies by su...,Another NEET aspirant died by suicide in Rajas...
2,Hampers to welcome MPs in new Parliament tomor...,In order to mark the first-ever working day of...
3,"Only 10% women lawmakers in RS, while only 14%...","Congress President Mallikarjun Kharge, while s..."
4,"Ganesh temple decorated with notes, coins wort...",The Sri Sathya Ganapathi Temple in Bengaluru a...


In [6]:
# Force conversion to string for specific columns, handling NaNs automatically
cols_to_clean = ['Headline', 'Content']

for col in cols_to_clean:
    data[col] = data[col].astype(str).str.lower()

# Verify the changes
data.head()

,Headline,Content
0,congress leader baljinder singh shot dead at h...,congress leader baljinder singh was shot dead ...
1,17-year-old girl preparing for neet dies by su...,another neet aspirant died by suicide in rajas...
2,hampers to welcome mps in new parliament tomor...,in order to mark the first-ever working day of...
3,"only 10% women lawmakers in rs, while only 14%...","congress president mallikarjun kharge, while s..."
4,"ganesh temple decorated with notes, coins wort...",the sri sathya ganapathi temple in bengaluru a...


In [7]:
# Add start and end signs to the headline column
def add_start_end_signs(Headline):
    return f"<start> {Headline} <end>"
data['Headline'] = data['Headline'].apply(add_start_end_signs)

In [8]:
data.head()

,Headline,Content
0,<start> congress leader baljinder singh shot d...,congress leader baljinder singh was shot dead ...
1,<start> 17-year-old girl preparing for neet di...,another neet aspirant died by suicide in rajas...
2,<start> hampers to welcome mps in new parliame...,in order to mark the first-ever working day of...
3,"<start> only 10% women lawmakers in rs, while ...","congress president mallikarjun kharge, while s..."
4,"<start> ganesh temple decorated with notes, co...",the sri sathya ganapathi temple in bengaluru a...


In [9]:
print(data['Headline'].apply(type).value_counts())

Headline
<class 'str'>    307696
Name: count, dtype: int64


In [10]:
data['Headline_len'] = data['Headline'].str.split().str.len()
data['Content_len'] = data['Content'].str.split().str.len()

data.head()

,Headline,Content,Headline_len,Content_len
0,<start> congress leader baljinder singh shot d...,congress leader baljinder singh was shot dead ...,13,60
1,<start> 17-year-old girl preparing for neet di...,another neet aspirant died by suicide in rajas...,13,59
2,<start> hampers to welcome mps in new parliame...,in order to mark the first-ever working day of...,12,60
3,"<start> only 10% women lawmakers in rs, while ...","congress president mallikarjun kharge, while s...",15,60
4,"<start> ganesh temple decorated with notes, co...",the sri sathya ganapathi temple in bengaluru a...,13,56


In [11]:
content_len = data['Content_len']
statistics_content = content_len.describe(percentiles=[0.90, 0.95, 0.98, 0.99])
headline_len = data['Headline_len']
statistics_headline = headline_len.describe(percentiles=[0.90, 0.95, 0.98, 0.99])

print(f'content{statistics_content}')
print(f'headline{statistics_headline}')

contentcount    307696.000000
mean         56.760410
std           4.029603
min          24.000000
50%          58.000000
90%          60.000000
95%          60.000000
98%          60.000000
99%          60.000000
max          61.000000
Name: Content_len, dtype: float64
headlinecount    307696.000000
mean         12.284141
std           2.244143
min           5.000000
50%          12.000000
90%          15.000000
95%          16.000000
98%          17.000000
99%          18.000000
max          23.000000
Name: Headline_len, dtype: float64


In [12]:
max_len_content = 60
max_len_headline = 16

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 25000
oov_token = '<nan>'
headline_list = data['Headline'].tolist()
content_list = data['Content'].tolist()
all_data = headline_list + content_list 
tokenizer = Tokenizer(num_words=num_words, oov_token = oov_token)

tokenizer.fit_on_texts(all_data)


In [14]:
x_sequences = tokenizer.texts_to_sequences(content_list)
y_sequences = tokenizer.texts_to_sequences(headline_list)

In [15]:
# Encoder (Bidirectional GRU)
import tensorflow as tf 
from tensorflow.keras.layers import Input, Embedding , GRU , Dense , Bidirectional 
enc_inputs = Input(shape=(max_len_content,), name='encoder_inputs')
enc_embedding = Embedding(input_dim=num_words, output_dim=128, mask_zero=True)(enc_inputs)
gru_layer = Bidirectional(GRU(256, return_sequences=True ,return_state=True , dropout=0.4 ))
enc_outputs , state_forward , state_backward = gru_layer(enc_embedding)

I0000 00:00:1768887656.023090   10443 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2158 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [16]:
# TheBridge code b/w GRU and LSTM (because LSTM has 2 states)
from tensorflow.keras.layers import Concatenate , Dense 

state_merged = Concatenate()([state_forward , state_backward])

# LSTM expects 2 states : hidden state and cell state
state_h = Dense(256, name='bridge_h')(state_merged)
state_c = Dense(256, name='bridge_c')(state_merged)

# packaging them for LSTM
encoder_states = [state_h , state_c]

In [17]:
#  Decoder (LSTM with Attention)
dec_inputs = Input(shape=(None,) , name = 'decoder_inputs')
dec_embedding_layer = Embedding(input_dim = num_words , output_dim = 128 , mask_zero=True , name='decoder_embedding')
decoder_embedding = dec_embedding_layer(dec_inputs)

# LSTM layer
from tensorflow.keras.layers import LSTM
decoder_lstm = LSTM(256 , return_sequences=True , return_state=True , name='decoder_lstm' ,dropout=0.4 , recurrent_dropout=0.4 )
decoder_outputs , _,_ = decoder_lstm(decoder_embedding , initial_state=encoder_states)

In [18]:
# Attention mechanism 
from tensorflow.keras.layers import AdditiveAttention
key_projector = Dense(256, name='key_projector')
enc_outputs_projected = key_projector(enc_outputs)

attention_layer = AdditiveAttention(name='attention_layer')
context_vector = attention_layer([decoder_outputs , enc_outputs , enc_outputs_projected])

In [19]:
# Decoder + Attention output
decoder_combined = Concatenate(axis=-1)([decoder_outputs , context_vector])

#  Output 
decoder_dense = Dense(num_words , activation='softmax' , name='output_layer')
output = decoder_dense(decoder_combined)

In [20]:
#  compile 
from tensorflow.keras.models import Model 
model = Model([enc_inputs , dec_inputs],output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 60)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 60, 128)   │  3,200,000 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 60)        │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 60, 512), │    592,896 │ embedding[0][0],  │
│ (Bidirectional)     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 128) │  3,200,000 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bridge_h (Dense)    │ (None, 256)       │    131,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bridge_c (Dense)    │ (None, 256)       │    131,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    394,240 │ decoder_embeddin… │
│                     │ 256), (None,      │            │ bridge_h[0][0],   │
│                     │ 256), (None,      │            │ bridge_c[0][0]    │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ key_projector       │ (None, 60, 256)   │    131,328 │ bidirectional[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_layer     │ (None, None, 512) │        256 │ decoder_lstm[0][… │
│ (AdditiveAttention) │                   │            │ bidirectional[0]… │
│                     │                   │            │ key_projector[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, None, 768) │          0 │ decoder_lstm[0][… │
│ (Concatenate)       │                   │            │ attention_layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, None,      │ 19,225,000 │ concatenate_1[0]… │
│ (Dense)             │ 25000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 27,006,376 (103.02 MB)

 Trainable params: 27,006,376 (103.02 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Data Slicing
x_sequences = pad_sequences(x_sequences, maxlen=60, padding='post')
y_sequences = pad_sequences(y_sequences, maxlen=16, padding='post')
x_sequences = np.array(x_sequences)
y_sequences = np.array(y_sequences)

dec_input_data = y_sequences[: ,:-1]
dec_target_data = y_sequences[: ,1:]

print(f"Original shape: {y_sequences.shape}")
print(f"Input shape:    {dec_input_data.shape}")
print(f"Target shape:   {dec_target_data.shape}")

Original shape: (307696, 16)
Input shape:    (307696, 15)
Target shape:   (307696, 15)


In [23]:
# Early stopping
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True , verbose=1)

# Checkpointing
checkpoint = ModelCheckpoint(
    'best_headline_generator.keras', # The file name
    monitor='val_loss',
    save_best_only=True,          # Only save if it's better than the last one
    verbose=1
)

In [24]:
# compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
#  model fit 
history = model.fit(
    [x_sequences , dec_input_data],
    dec_target_data,
    batch_size=16,
    epochs=20,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/20


2026-01-20 11:12:00.734466: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2026-01-20 11:12:01.907433: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701


15385/15385 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.3263 - loss: 4.8301
Epoch 1: val_loss improved from None to 5.41732, saving model to best_headline_generator.keras
15385/15385 ━━━━━━━━━━━━━━━━━━━━ 1369s 89ms/step - accuracy: 0.4505 - loss: 3.5636 - val_accuracy: 0.2749 - val_loss: 5.4173
Epoch 2/20
15385/15385 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.6369 - loss: 1.9506
Epoch 2: val_loss did not improve from 5.41732
15385/15385 ━━━━━━━━━━━━━━━━━━━━ 1364s 89ms/step - accuracy: 0.6517 - loss: 1.8567 - val_accuracy: 0.2872 - val_loss: 5.4693
Epoch 3/20
15385/15385 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.7053 - loss: 1.4603
Epoch 3: val_loss did not improve from 5.41732
15385/15385 ━━━━━━━━━━━━━━━━━━━━ 1360s 88ms/step - accuracy: 0.7093 - loss: 1.4459 - val_accuracy: 0.2934 - val_loss: 5.5752
Epoch 3: early stopping
Restoring model weights from the end of the best epoch: 1.


In [26]:
# Machine 1: The Encoder
# We take the Article Input (enc_inputs)
# We return:
#   1. enc_outputs (The Map of the article)
#   2. state_h (The final thought from the Bridge)
#   3. state_c (The final memory from the Bridge)

# Since you already defined these variables in Cell 14 & 15, we just group them:
encoder_model = Model(inputs=enc_inputs, outputs=[enc_outputs, state_h, state_c])

print("Encoder Model Created!")
encoder_model.summary()

Encoder Model Created!


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 60)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 60, 128)   │  3,200,000 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 60)        │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 60, 512), │    592,896 │ embedding[0][0],  │
│ (Bidirectional)     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bridge_h (Dense)    │ (None, 256)       │    131,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bridge_c (Dense)    │ (None, 256)       │    131,328 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,055,552 (15.47 MB)

 Trainable params: 4,055,552 (15.47 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.models import Model

# =============================
# 1. DEFINE NEW INPUTS
# =============================
# A single word (e.g., "apple")
inf_dec_input = Input(shape=(1,), name='inf_dec_input')

# The Encoder's Map (we pass this in every step so Attention can scan it)
inf_enc_map = Input(shape=(60, 512), name='inf_enc_map')

# The Past States (from the previous step)
inf_state_h = Input(shape=(256,), name='inf_h_input')
inf_state_c = Input(shape=(256,), name='inf_c_input')
dec_states_inputs = [inf_state_h, inf_state_c]

# =============================
# 2. REUSE YOUR TRAINED LAYERS
# =============================
# A. Embedding
# Reuse 'dec_embedding_layer' from Cell 16
inf_embedding = dec_embedding_layer(inf_dec_input)

# B. LSTM
# Reuse 'decoder_lstm' from Cell 16
# We pass the past states as 'initial_state'
inf_dec_outputs, state_h_out, state_c_out = decoder_lstm(
    inf_embedding, initial_state=dec_states_inputs
)

# C. Attention
# Reuse 'key_projector' from Cell 26
# We must project the new map input to match dimensions
inf_projected_map = key_projector(inf_enc_map)

# Reuse 'attention_layer' from Cell 26
# Query=Decoder, Value=Map, Key=Projected Map
inf_context = attention_layer([inf_dec_outputs, inf_enc_map, inf_projected_map])

# D. Output
# Concatenate context and output
inf_combined = Concatenate(axis=-1)([inf_dec_outputs, inf_context])

# Reuse 'decoder_dense' from Cell 27
inf_probs = decoder_dense(inf_combined)

# =============================
# 3. CREATE THE MODEL
# =============================
decoder_model = Model(
    inputs=[inf_dec_input, inf_enc_map, inf_state_h, inf_state_c],
    outputs=[inf_probs, state_h_out, state_c_out]
)

print("Decoder Model Created!")
decoder_model.summary()

Decoder Model Created!


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inf_dec_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 1, 128)    │  3,200,000 │ inf_dec_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inf_h_input         │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inf_c_input         │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inf_enc_map         │ (None, 60, 512)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 1, 256),  │    394,240 │ decoder_embeddin… │
│                     │ (None, 256),      │            │ inf_h_input[0][0… │
│                     │ (None, 256)]      │            │ inf_c_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ key_projector       │ (None, 60, 256)   │    131,328 │ inf_enc_map[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_layer     │ (None, 1, 512)    │        256 │ decoder_lstm[1][… │
│ (AdditiveAttention) │                   │            │ inf_enc_map[0][0… │
│                     │                   │            │ key_projector[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 1, 768)    │          0 │ decoder_lstm[1][… │
│ (Concatenate)       │                   │            │ attention_layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 1, 25000)  │ 19,225,000 │ concatenate_3[0]… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,950,824 (87.55 MB)

 Trainable params: 22,950,824 (87.55 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Create the Dictionary to turn Numbers back into Words
reverse_word_index = tokenizer.index_word

In [29]:
def decode_sequence(input_seq):
    # 1. ENCODE: Read the article once
    # the "Map" (enc_out) and "Initial Thoughts" (state_h, state_c)
    enc_out, state_h, state_c = encoder_model.predict(input_seq, verbose=0)

    #  Prepare the first input
    # container for the input word (Shape: 1x1)
    target_seq = np.zeros((1, 1))
    # the <start> token ID
    target_seq[0, 0] = tokenizer.word_index['start']

    stop_condition = False
    decoded_sentence = ""

    # 3.GENERATION LOOP
    while not stop_condition:
        # A. Predict the next word
        # We feed: [Current Word] + [Article Map] + [Past States]
        output_tokens, h, c = decoder_model.predict(
            [target_seq, enc_out, state_h, state_c], 
            verbose=0
        )

        # B. Sample the best token (Argmax)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Convert ID to Word (Handle case where ID might be 0)
        sampled_char = reverse_word_index.get(sampled_token_index, '')

        # C. Add to the result string
        if sampled_char != 'end':
            decoded_sentence += ' ' + sampled_char

        # D. Check if we should stop
        # Stop if we hit '<end>' OR the headline gets too long (e.g. > 20 words)
        if (sampled_char == 'end' or len(decoded_sentence.split()) > 20):
            stop_condition = True

        # E. Update for the next loop (The Relay Race Handover)
        # The predicted word becomes the NEXT input
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # The new states become the PAST states
        state_h, state_c = h, c

    return decoded_sentence.strip()

In [30]:
# Pick a random index
idx = np.random.randint(0, len(x_sequences))

# Get the data
input_seq = x_sequences[idx:idx+1]
original_text = tokenizer.sequences_to_texts(input_seq)[0]
original_headline = tokenizer.sequences_to_texts(y_sequences[idx:idx+1])[0]

# Generate!
generated_headline = decode_sequence(input_seq)

print("Original Article Snippet:", original_text[:100], "...")
print("Real Headline: ", original_headline)
print("Generated Headline:", generated_headline)

Original Article Snippet: the origin story of how a <nan> came to be an integral part of our makeup arsenal is history with pe ...
Real Headline:  start winter blush is having its moment and here are our favourites end <nan> <nan> <nan>
Generated Headline: winter blush is having the moment here is having here


In [34]:
# Save the Trained Models 

import pickle 

# Inference Models 
encoder_model.save('encoder_model.keras')
decoder_model.save("decoder_model.keras")

# Save the tokenizer to freeze the dictionary

with open('tokenizer.pkl','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

print("All the files are saved.")

All the files are saved.
